In [1]:
import pandas as pd
import numpy as np
import flip_data
import gd_poisoner
import datetime

In [2]:
poi_train_x = pd.read_csv('train_X.csv')
print("ori train ", poi_train_x.shape[0])
poi_train_x = np.matrix(poi_train_x.to_numpy())
print("matrix train ", len(poi_train_x))
poi_train_y = pd.read_csv('train_y.csv')
print("ori train y ", poi_train_y.shape[0])
poi_train_y = poi_train_y['Life Expectancy'].tolist()
print("matrix train y ", len(poi_train_y))
poi_test_x = pd.read_csv('test_X.csv')
poi_test_x = np.matrix(poi_test_x.to_numpy())
poi_test_y = pd.read_csv('test_y.csv')
poi_test_y = poi_test_y['Life Expectancy'].tolist()
poi_val_x = pd.read_csv('val_X.csv')
poi_val_x = np.matrix(poi_val_x.to_numpy())
poi_val_y = pd.read_csv('val_y.csv')
poi_val_y = poi_val_y['Life Expectancy'].tolist()

print (poi_train_x[0])
print (poi_train_y[0])

# num of poisoning points
poison_ct = 300
# num of points to train model
train_ct = len(poi_train_x)
test_ct = len(poi_test_x)
# proportion of poisoning
total_prop = poison_ct / (poison_ct + train_ct)
print ("Posion count: ", poison_ct, " Train count: ", train_ct, " Test count: ", test_ct, " Proportion of poisoning: ", total_prop)

x_pois, y_pois = flip_data.flip_data.perform_flip(poi_train_x,poi_train_y,poison_ct)

eta = 0.01 # gradient descent step size (note gradients are normalized)
beta = 0.05 # decay rate for line search
sigma = 0.9 # line search stop condition
epsilon = 1e-3 # poisoning stop condition
trainfile = open("train.txt", 'w')
resfile = open("err.txt", 'w')
colmap = {}
poiser = gd_poisoner.lasso_poisoner(poi_train_x, poi_train_y, poi_test_x, poi_test_y, poi_val_x, poi_val_y,\
                                  eta, beta, sigma, epsilon, trainfile, resfile,colmap)
clf, _ = poiser.learn_model(np.concatenate((poi_train_x, x_pois), axis=0), poi_train_y + y_pois, None)
print("clf: ", clf)
err = poiser.computeError(clf)[0]
x_pois = np.matrix(x_pois)
y_pois = y_pois[:]
for i in  range(5):
    curprop = (i + 1)*total_prop/5
    numsamples = int(0.5 + train_ct*(curprop/(1 - curprop)))
    curpoisx = x_pois[:numsamples,:]
    curpoisy = y_pois[:numsamples]
    trainfile.write("\n")
    timestart = datetime.datetime.now()
    newlogdir = 'a.jpg'
    poisres, poisresy = poiser.poison_data(curpoisx, curpoisy, timestart, False, newlogdir)
    print(poisres.shape,poi_train_x.shape)
    poisedx = np.concatenate((poi_train_x,poisres),axis = 0)
    poisedy = poi_train_y + poisresy
    clfp, _ = poiser.learn_model(poisedx,poisedy,None)
    clf = poiser.initclf
    print("poisres len: ",len(poisres),", poisresy len: ",len(poisresy),", poisedx len: ",len(poisedx),", poisedy len: ",len(poisedy))
    print("clfp ",clfp,", clf2 ",clf)
    print(" ")
    errgrd = poiser.computeError(clf)
    err = poiser.computeError(clfp)

print("poi_train_x ", len(poi_train_x),\
            ", poi_train_y ", len(poi_train_y),\
            ", poi_test_x", len(poi_test_x),\
            ", poi_test_y", len(poi_test_y),\
            ", poi_val_x", len(poi_val_x),\
            ", poi_val_y", len(poi_val_y))
print("")
print("Unpoisoned")
print("Validation MSE:", errgrd[0])
print("Test MSE:", errgrd[1])
print('Poisoned:')
print("Validation MSE:", err[0])
print("Test MSE:", err[1])

ori train  1319
matrix train  1319
ori train y  1319
matrix train y  1319
[[2.00900000e+03 0.00000000e+00 2.52077562e-01 2.22222222e-03
  1.14221725e-01 6.87422107e-04 9.79591837e-01 9.89711711e-05
  2.10892236e-01 1.60000000e-03 9.79166667e-01 1.74114916e-01
  9.79381443e-01 0.00000000e+00 1.80413125e-03 1.54066216e-05
  5.50724638e-01 5.36842105e-01 7.75316456e-01 6.47342995e-01]]
0.6736242884250474
Posion count:  300  Train count:  1319  Test count:  165  Proportion of poisoning:  0.1852995676343422
y_opp [0. 0. 0. ... 0. 0. 0.]
stats [0. 0. 0. ... 0. 0. 0.]
total_prob 132.78026565464893
x_pois len:  300
x_pois col ct: 20
y_pois len:  300
clf:  Lasso(alpha=3.7162432899202766e-05, max_iter=10000, warm_start=True)
Poison Count: 51
Iteration 0:
Objective Value: 0.00691527993399966 Change: 0.00691527993399966
Validation MSE: 0.005485820133364701
Test MSE: 0.007423407794221996
Iteration 1:
Objective Value: 0.005012248368850984 Change: -0.0019030315651486756
Validation MSE: 0.004161851148